# 08b. Using the quick deployment app - remote part

## Overview

This notebook is intended to be executed on the cluster as a continuation of notebook

```
08a-Using_the_quick_deployment_app_-_local_part.ipynb
```.

## Import idact

We will use a wildcard import for convenience:

In [1]:
from idact import *

## Load the cluster

Let's load the environment and the cluster. Make sure to use your cluster name.

In [2]:
load_environment()
cluster = show_cluster("hpc")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='/net/people/plggarstka/.ssh/id_rsa_cm', install_key=False, disable_sshd=False)

In [3]:
node = cluster.get_access_node()
node

Node(pro.cyfronet.pl:22, None)

In [4]:
node.connect()

## Pull deployments

You can now access the deployments `idact-notebook` created for you:

In [5]:
deployments = cluster.pull_deployments()
deployments

2018-11-24 21:34:48 INFO: Pulling deployments.
2018-11-24 21:34:50 INFO: Creating the ssh directory.
2018-11-24 21:35:01 INFO: Pulled allocation deployment: Nodes([Node(p0613:44654, 2018-11-24 20:53:36.191631+00:00),Node(p0629:34396, 2018-11-24 20:53:36.191631+00:00),Node(p0630:41649, 2018-11-24 20:53:36.191631+00:00)], SlurmAllocation(job_id=14337694))
2018-11-24 21:35:01 INFO: Pulled Jupyter deployment: JupyterDeployment(8080 -> Node(p0613:44654, 2018-11-24 20:53:36.191631+00:00)


SynchronizedDeployments(nodes=1, jupyter_deployments=1, dask_deployments=0)

There is the nodes deployment:

In [6]:
nodes = deployments.nodes[-1]
nodes

Nodes([Node(p0613:44654, 2018-11-24 20:53:36.191631+00:00),Node(p0629:34396, 2018-11-24 20:53:36.191631+00:00),Node(p0630:41649, 2018-11-24 20:53:36.191631+00:00)], SlurmAllocation(job_id=14337694))

And the Jupyter deployment this notebook is running on:

In [7]:
nb = deployments.jupyter_deployments[-1]
nb

JupyterDeployment(8080 -> Node(p0613:44654, 2018-11-24 20:53:36.191631+00:00)

## Deploy Dask

You can deploy Dask from the remote notebook, though you won't be able to browse the dashboards this way:

In [8]:
dd = deploy_dask(nodes)
dd

2018-11-24 21:35:04 INFO: Deploying Dask on 3 nodes.
2018-11-24 21:35:04 INFO: Connecting to p0613:44654 (1/3).
2018-11-24 21:35:06 INFO: Connecting to p0629:34396 (2/3).
2018-11-24 21:35:07 INFO: Connecting to p0630:41649 (3/3).
2018-11-24 21:35:08 INFO: Deploying scheduler on the first node: p0613.
2018-11-24 21:35:20 INFO: Desired local tunnel port 35898 is taken. Binding to random port instead.
2018-11-24 21:35:21 INFO: Desired local tunnel port 34102 is taken. Binding to random port instead.
2018-11-24 21:35:28 INFO: Checking scheduler connectivity from p0613 (1/3).
2018-11-24 21:35:28 INFO: Checking scheduler connectivity from p0629 (2/3).
2018-11-24 21:35:28 INFO: Checking scheduler connectivity from p0630 (3/3).
2018-11-24 21:35:28 INFO: Deploying workers.
2018-11-24 21:35:28 INFO: Deploying worker 1/3.
2018-11-24 21:35:40 INFO: Desired local tunnel port 50383 is taken. Binding to random port instead.
2018-11-24 21:35:41 INFO: Deploying worker 2/3.
2018-11-24 21:35:49 INFO: Dep

DaskDeployment(scheduler=tcp://localhost:39783/tcp://172.20.66.103:35898, workers=3)

In [9]:
client = dd.get_client()
client

Client Scheduler: tcp://localhost:39783 Dashboard: http://localhost:34102/status,Cluster Workers: 3 Cores: 6 Memory: 32.21 GB


In [10]:
x = client.submit(lambda value: value + 1, 10)

In [11]:
x.result() == 11

True

In [12]:
client.close()

## Cancel the allocation (recommended)

After you're done, you should cancel the nodes from the remote notebook, in order to avoid being charged for additional resources.

You will lose the connection to this notebook straight away, so make sure to save beforehand.

In [ ]:
nodes.cancel()

## Continue with the local notebook

Perform the rest of instructions in the local notebook.